In [47]:
# Importing important libraries and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

In [48]:
df=pd.read_csv("/content/labeledTrainData.tsv", sep='\t')

In [49]:
df

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...


Data Preprocessing:

In [50]:
# Cheking for any missing values in training data
df.isnull().sum().sum()

0

In [51]:
# Since we have three columns in which the sentiment column is target variable and id is not required for training so we will pre process the training data containing reviews only
# Each review is in the form of a paragraph with HTML tags, punctuations, numbers and stopwords such as is, are am etc so we need to clean them
# For cleaning below is the function which uses beautifulsoup, regular expression and natural language toolkit to achieve the above

# creating y as target variable in the form of an array
y = np.array(df["sentiment"])

# To eliminate stop word we need to download its vacab from nltk
nltk.download('stopwords')

def process(review):
   # review without HTML tags
   review = BeautifulSoup(review).get_text()
   # review without punctuation and numbers
   review = re.sub("[^a-zA-Z]",' ',review)
   # converting into lowercase and splitting to eliminate stopwords
   review = review.lower()
   review = review.split()
   # review without stopwords
   swords = set(stopwords.words("english"))                      # conversion into set for fast searching
   review = [w for w in review if w not in swords]
   # joining of splitted paragraph by spaces and return
   return(" ".join(review))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
# processing the training data with the help of function defined above

x = []
for r in range(len(df["review"])):
  x.append(process(df["review"][r]))

<ipython-input-51-00f0df38f10a>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review = BeautifulSoup(review).get_text()


Modeling using Bag of Words:

In [53]:
# Now we have our processed and cleaned training set but it is in the form of text but to train we need to convert it into numerical data
# For that we will use bag of words which is based on the frequency that each word occur in a review within training data and we will go for 5000 most common words

# Initializing the countvectorizer which is a sklearn tool for bag of words
vectorizer = CountVectorizer( max_features = 5000 )
# Now we will use fit_transform which fits the model to learn vocabulary for 5000 most common words and then transform the training data into feature vectors
x = vectorizer.fit_transform(x)
# conversion into array
x = x.toarray()

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [55]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=1,stratify=y)

In [56]:
#Naive Bayes Model

from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
nb.fit(x_train,y_train)
MultinomialNB()
train_pred=nb.predict(x_train)
test_pred=nb.predict(x_test)
print(classification_report(test_pred,y_test))

              precision    recall  f1-score   support

           0       0.86      0.84      0.85      2560
           1       0.83      0.85      0.84      2440

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



In [57]:
#Random Forest Model

from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(x_train,y_train)
RandomForestClassifier()
train_pred=rfc.predict(x_train)
test_pred=rfc.predict(x_test)
print(classification_report(test_pred,y_test))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84      2565
           1       0.82      0.85      0.83      2435

    accuracy                           0.84      5000
   macro avg       0.84      0.84      0.84      5000
weighted avg       0.84      0.84      0.84      5000



In [58]:
from sklearn.svm import LinearSVC
SVCmodel = LinearSVC()
SVCmodel.fit(x_train, y_train)
LinearSVC()
train_pred=SVCmodel.predict(x_train)
test_pred = SVCmodel.predict(x_test)
print(classification_report(test_pred,y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.84      0.83      0.83      2514
           1       0.83      0.84      0.83      2486

    accuracy                           0.83      5000
   macro avg       0.83      0.83      0.83      5000
weighted avg       0.83      0.83      0.83      5000



In [59]:
from sklearn.linear_model import LogisticRegression
LRmodel = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
LRmodel.fit(x_train, y_train)
LogisticRegression()
train_pred=LRmodel.predict(x_train)
test_pred = LRmodel.predict(x_test)
print(classification_report(test_pred,y_test))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      2519
           1       0.85      0.85      0.85      2481

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



Naive bayes and Logistic Regression shows better result

Modeling using LSTM

In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [16]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df['Processed_Reviews'] = df.review.apply(lambda x: clean_text(x))

In [17]:
df

,id,sentiment,review,Processed_Reviews
0,5814_8,1,With all this stuff going down at the moment w...,stuff go moment mj ive start listen music watc...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",classic war world timothy hines entertain film...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,film start manager nicholas bell give welcome ...
3,3630_4,0,It must be assumed that those who praised this...,must assume praise film greatest film opera ev...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious 80 ex...
...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,seem like consideration ha go imdb review film...
24996,5064_1,0,I don't believe they made this film. Completel...,dont believe make film completely unnecessary ...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...",guy loser cant get girl need build pick strong...
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,30 minute documentary buñuel make early 1930s ...


In [44]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers


max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(df['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df['sentiment']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/3
200/200 [==============================] - 35s 148ms/step - loss: 0.4398 - accuracy: 0.7942 - val_loss: 0.3244 - val_accuracy: 0.8666
Epoch 2/3
200/200 [==============================] - 16s 81ms/step - loss: 0.2478 - accuracy: 0.9025 - val_loss: 0.3246 - val_accuracy: 0.8662
Epoch 3/3
200/200 [==============================] - 12s 60ms/step - loss: 0.1849 - accuracy: 0.9314 - val_loss: 0.3455 - val_accuracy: 0.8606
